# Toyota Sienna Inventory Guide

This notebook helps you find available and upcoming Toyota Siennas based on your preferred states and models.

**How to use:**

1.  **Specify your preferences:** In the first code cell, update the `states` and `models` lists with your desired states and Sienna models.
2.  **Run all cells:** Execute all the code cells in the notebook.
3.  **View the results:** The notebook will display summaries of available and upcoming Siennas based on your filters, highlighting entries with minimal options.

**Available Options:**

*   **Models:** 'LE FWD', 'LE AWD', 'XSE FWD', 'XSE AWD', 'Woodland Edition AWD', 'Platinum FWD', 'Platinum AWD', 'XLE FWD', 'XLE AWD', 'Limited FWD', 'Limited AWD'
*   **Colors:** 'Magnetic Gray Metallic', 'Ruby Flare Pearl', 'Midnight Black Metallic', 'Cypress', 'Ice Cap', 'Celestial Silver Metallic', 'Blueprint', 'Predawn Gray Mica', 'Cement', 'Wind Chill Pearl'
*   **Interior Colors:** 'Gray Woven Fabric', 'Black leather-trimmed', 'Black Softex', 'Cool Gray Softex', 'Macadamia leather-trimmed', 'Nobel Brown Leather', 'Gray Softex', 'Graphite Leather'

**Data Source:**

The data used in this guide is sourced from this Google Drive location [Vehicle_Inventory](https://drive.google.com/drive/u/0/folders/1uOnNR7wVHN6o5rTjcMhVsNL3SKq5UJwC), which is updated daily. A big thank you to the individuals who diligently upload the CSV file every day, making this notebook possible!

In [1]:
# Define the states and models to filter by
states = ['CA']
models = ['Limited FWD', 'Limited AWD', 'Platinum FWD', 'Platinum AWD']
colors = ['Ruby Flare Pearl', 'Wind Chill Pearl']                                           # keep empty [] to show all colors
interior_colors = ['Macadamia leather-trimmed', 'Black leather-trimmed']                    # keep empty [] to show all colors
zip_code = 94102
entries_to_display = 20


In [2]:
!pip install pgeocode -q
import pgeocode
import pandas as pd


print(f"\n📍 Looking up ZIP code: {zip_code}")

# Initialize US geocoder
nomi = pgeocode.Nominatim('us')

# Get location info
location = nomi.query_postal_code(zip_code)

# Check if valid
if pd.isna(location.latitude):
    print(f"❌ Invalid or not found: {zip_code}")
else:
    print(f"✅ Location found!")
    print(f"   Place: {location.place_name}")
    print(f"   State: {location.state_code}")
    print(f"   County: {location.county_name}")
    print(f"   Latitude: {location.latitude}")
    print(f"   Longitude: {location.longitude}")

    # Store coordinates
    user_lat = location.latitude
    user_long = location.longitude


📍 Looking up ZIP code: 94102
✅ Location found!
   Place: San Francisco
   State: CA
   County: City and County of San Francisco
   Latitude: 37.7813
   Longitude: -122.4167


In [3]:
import ipywidgets as widgets
from IPython.display import display

# URLs for the inventory
inventory_url = 'https://docs.google.com/spreadsheets/d/1K43yTr2wGy7QV4kqVESRlkZZKY8i2VM9/export?format=csv' ## sienna inventory

# Read the data from the URLs into pandas DataFrames
df = pd.read_csv(inventory_url)

# Filter the DataFrame based on the specified states
df = df[df['Dealer State'].isin(states)]

if models:
  df = df[df['Model'].isin(models)]

if colors:
    df = df[df['Color'].isin(colors)]
if interior_colors:
    df = df[df['Int Color'].isin(interior_colors)]

# Calculate 'Option Price' for both DataFrames using .loc for explicit indexing
# 1495 is the delivery charge
df['Option Price'] = df['Total MSRP'] - df['Base MSRP'] - 1495

# Create 'Dealer Location' column by merging 'Dealer City' and 'Dealer State'
df['Dealer Location'] = df['Dealer City'] + ', ' + df['Dealer State']

# Calculate distance from user's location to dealer
# Ensure user_lat and user_long are defined (from the previous cell)
if 'user_lat' in locals() and 'user_long' in locals():
    from geopy.distance import geodesic
    df['Distance'] = df.apply(
        lambda row: round(geodesic((user_lat, user_long), (row['Dealer Lat'], row['Dealer Long'])).miles),
        axis=1
    )
else:
    df['Distance'] = None
    print("User location not available. Please run the cell to enter your zip code.")


# Sort the DataFrames by 'Selling Price'
df = df.sort_values(by=['Selling Price'])

available_df = df[df['Shipping Status'] == 'At dealer']
upcoming_df = df[df['Shipping Status'].isin(['Factory to port', 'Port to dealer'])]

# uncomment to Display a sample row

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)
display(df.head(1))

,Year,Model,Color,Int Color,Base MSRP,Total MSRP,Selling Price,Selling Price Incomplete,Markup,TMSRP plus DIO,Shipping Status,Pre-Sold,Hold Status,eta.currFromDate,eta.currToDate,VIN,isTempVin,Dealer,Dealer Website,Dealer State,Dealer City,Dealer Zip,Dealer Lat,Dealer Long,CenterLat,CenterLong,DistanceFromCenter,Transmission,MPG Combined,MPG City,MPG Highway,Engine Code,Engine Name,Cab Code,Cab,Bed Code,Bed,FirstAddedDate,LastChangedDateTime,infoDateTime,Options,Option Price,Dealer Location,Distance
9754,2025,Limited FWD,Wind Chill Pearl,Black leather-trimmed,50500,53739,53739.0,False,0.0,53739,Factory to port,False,Available,2025-10-23,2025-11-07,5TDZRKEC5SS42C541,True,Toyota Carlsbad,https://www.toyotacarlsbad.com,CA,Carlsbad,92008,33.132685,-117.326399,41.978382,-91.668626,NaN,Electronically controlled Continuously Variabl...,36.0,36.0,36.0,2HE,2.5L Hybrid Engine,NaN,NaN,NaN,NaN,2025-09-11 1:26:16,2025-10-04 2:00:56,2025-10-04 2:00:56,1500W inverter | 50 State Emissions | All Weat...,1744,"Carlsbad, CA",430


In [4]:
def highlight_red(row):
    """Highlights rows with markup greater than 1000 in red."""
    styles = [''] * len(row)
    if row['Markup'] > 1000:
        styles = ['background-color: #ffcccc'] * len(row) ## markup
    return styles

# def highlight_yellow(row):
#     """Highlights rows with specific colors in yellow."""
#     styles = [''] * len(row)
#     if row['Color'] in ['Celestial Silver Metallic', 'Blueprint', 'Magnetic Gray Metallic']: ## dont like these colors
#         styles = ['background-color: #ffff99'] * len(row)
#     return styles

def highlight_green(row):
    """Highlights rows with low option price and negative markup in green."""
    styles = [''] * len(row)
    if row['Option Price'] <= 1500 and row['Markup'] <= 0:
        styles = ['background-color: #90ee90'] * len(row)
    return styles

def display_summary(df):
  """Highlights inventory based on specific criteria.

  Args:
    df: The input pandas DataFrame.

  Returns:
    A Styler object with highlighting applied.
  """
  df_display = df[['VIN', 'Model', 'Color', 'Int Color', 'Base MSRP', 'Total MSRP', 'Option Price', 'Selling Price', 'Markup', 'Dealer', 'Dealer Location', 'Distance', 'FirstAddedDate', 'Options']]
  styler = df_display.style.apply(highlight_red, axis=1)
  #styler = styler.apply(highlight_yellow, axis=1)
  styler = styler.apply(highlight_green, axis=1)
  return styler

In [5]:
from IPython.display import Markdown

display(Markdown(f'# Toyota Sienna (In Stock)\n\n'))
display(Markdown(f'States: **{", ".join(states)}**'))
display(Markdown(f'Models: **{", ".join(models)}**'))
display(Markdown(f'Colors: **{", ".join(colors)}**'))


display_summary(available_df.head(entries_to_display))

# Toyota Sienna (In Stock)



States: **CA**

Models: **Limited FWD, Limited AWD, Platinum FWD, Platinum AWD**

Colors: **Ruby Flare Pearl, Wind Chill Pearl**

,VIN,Model,Color,Int Color,Base MSRP,Total MSRP,Option Price,Selling Price,Markup,Dealer,Dealer Location,Distance,FirstAddedDate,Options
9732,5TDZRKEC4SS283126,Limited FWD,Wind Chill Pearl,Black leather-trimmed,50500,55169,3174,56669.000000,1500.000000,Crown Toyota,"Ontario, CA",375,2025-09-18 1:26:02,1500W inverter | 50 State Emissions | All Weather Floor Liners | Alloy Wheel Locks | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | MAX SHIELD | Mudguards | Premium Paint | Rear Bumper Applique
10176,5TDZSKFC5SS216618,Limited AWD,Ruby Flare Pearl,Black leather-trimmed,52500,56730,2735,56730.000000,0.000000,Thurston Toyota,"Ukiah, CA",106,2025-09-18 1:24:53,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Premium Paint | Spare tire (removes second row ottomans)
10037,5TDZSKFC3SS202751,Limited AWD,Wind Chill Pearl,Black leather-trimmed,52200,56779,3084,56779.000000,0.000000,Temecula Valley Toyota,"Temecula, CA",417,2025-07-03 1:03:02,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Mudguards | Premium Paint | Rear Bumper Applique
9968,5TDZSKFC2SS216494,Limited AWD,Wind Chill Pearl,Macadamia leather-trimmed,52500,57175,3180,57175.000000,0.000000,Ventura Toyota,"Ventura, CA",303,2025-09-18 1:25:38,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Mudguards | Premium Paint | Tri-Fold Cargo Liner
10444,5TDZSKFC9SS215455,Limited AWD,Wind Chill Pearl,Black leather-trimmed,52500,57434,3439,57434.000000,0.000000,Toyota of Hemet,"Hemet, CA",411,2025-09-13 2:04:16,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Illuminated Door Sills | Premium Paint | Rear Bumper Applique | Tri-Fold Cargo Liner
9758,5TDZRKEC6SS281040,Limited FWD,Ruby Flare Pearl,Black leather-trimmed,50500,55483,3488,59778.000000,4295.000000,Claremont Toyota,"Claremont, CA",366,2025-09-06 5:58:06,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Cargo Net-Envelope with Pouch | CT protection pkg | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Illuminated Door Sills | Mudguards | Premium Paint | Rear Bumper Applique
10107,5TDZSKFC4SS215945,Limited AWD,Wind Chill Pearl,Black leather-trimmed,52500,60483,6488,60483.000000,0.000000,Dalton Toyota National City,"National City, CA",463,2025-09-16 1:29:47,1500W inverter | 50 State Emissions | All Weather Floor Liners for models w/ Vacuum and FridgeBox Package | Cargo Cross Bars | Cargo Net-Envelope with Pouch | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | FridgeBox and Vacuum Package | Illuminated Door Sills | Mudguards | Premium Paint | Rear Bumper Applique
10249,5TDZSKFC6SS214182,Limited AWD,Ruby Flare Pearl,Black leather-trimmed,52500,55759,1764,60754.000000,4995.000000,Santa Cruz Toyota,"Capitola, CA",61,2025-09-06 5:56:35,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Door Sill Protectors | Mudguards | Owner's Portfolio | Premium Paint | Rear Bumper Applique
9967,5TDZSKFC2SS215894,Limited AWD,Wind Chill Pearl,Macadamia leather-trimmed,52500,56939,2944,61018.000000,4079.000000,Toyota of Huntington Beach,"Huntington Beach, CA",376,2025-09-16 1:29:26,"1500W inverter | 50 State Emissions | All Weather Floor Liners | Cilajet Aviation Protection Package | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Mudguards | Premium Paint | Quick Charging Cable Packag

In [6]:
display(Markdown(f'# Toyota Sienna (Upcoming)\n\n'))
display(Markdown(f'States: **{", ".join(states)}**'))
display(Markdown(f'Models: **{", ".join(models)}**'))
display(Markdown(f'Colors: **{", ".join(colors)}**'))

display_summary(upcoming_df.head(entries_to_display))

# Toyota Sienna (Upcoming)



States: **CA**

Models: **Limited FWD, Limited AWD, Platinum FWD, Platinum AWD**

Colors: **Ruby Flare Pearl, Wind Chill Pearl**

,VIN,Model,Color,Int Color,Base MSRP,Total MSRP,Option Price,Selling Price,Markup,Dealer,Dealer Location,Distance,FirstAddedDate,Options
9754,5TDZRKEC5SS42C541,Limited FWD,Wind Chill Pearl,Black leather-trimmed,50500,53739,1744,53739.000000,0.000000,Toyota Carlsbad,"Carlsbad, CA",430,2025-09-11 1:26:16,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Mudguards | Premium Paint | Rear Bumper Applique | Spare tire
9793,5TDZRKEC8SS43D937,Limited FWD,Wind Chill Pearl,Macadamia leather-trimmed,50500,54735,2740,54735.000000,0.000000,Toyota Carlsbad,"Carlsbad, CA",430,2025-09-20 1:43:32,1500W inverter | 50 State Emissions | All Weather Floor Liners | Alloy Wheel Locks | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Premium Paint
9789,5TDZRKEC8SS43C741,Limited FWD,Wind Chill Pearl,Black leather-trimmed,50500,54800,2805,54800.000000,0.000000,Toyota of Orange,"Orange, CA",376,2025-09-20 1:44:16,1500W inverter | 50 State Emissions | All Weather Floor Liners | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Mudguards | Premium Paint
9691,5TDZRKEC1SS284444,Limited FWD,Wind Chill Pearl,Black leather-trimmed,50500,54804,2809,54804.000000,0.000000,South Bay Toyota,"Gardena, CA",355,2025-09-26 2:26:27,1500W inverter | 50 State Emissions | All Weather Floor Liners | Alloy Wheel Locks | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Premium Paint | Rear Bumper Applique
9749,5TDZRKEC5SS283328,Limited FWD,Wind Chill Pearl,Black leather-trimmed,50500,54924,2929,54924.000000,0.000000,Penske Toyota,"Downey, CA",358,2025-09-19 2:56:37,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Premium Paint | Rear Bumper Applique
9731,5TDZRKEC4SS283000,Limited FWD,Wind Chill Pearl,Black leather-trimmed,50500,55154,3159,55154.000000,0.000000,Temecula Valley Toyota,"Temecula, CA",417,2025-09-17 1:44:10,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Mudguards | Premium Paint | Rear Bumper Applique | Spare tire
9781,5TDZRKEC7SS43E139,Limited FWD,Wind Chill Pearl,Black leather-trimmed,50500,55174,3179,55174.000000,0.000000,Toyota Place,"Garden Grove, CA",373,2025-09-21 1:49:13,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Door Sill Protectors | Entertainment Package | Mudguards | Premium Paint | Rear Bumper Applique
9849,5TDZSKFC0SS220043,Limited AWD,Wind Chill Pearl,Black leather-trimmed,52500,55230,1235,55230.000000,0.000000,South Bay Toyota,"Gardena, CA",355,2025-10-03 1:48:30,1500W inverter | 50 State Emissions | All Weather Floor Liners | Digital rearview mirror with HomeLink universal transceiver | Premium Paint
9784,5TDZRKEC8SS42C808,Limited FWD,Ruby Flare Pearl,Black leather-trimmed,50500,55314,3319,55314.000000,0.000000,AutoNation Toyota Irvine,"Irvine, CA",388,2025-09-06 9:52:26,1500W inverter | 50 State Emissions | All Weather Floor Liners | Alloy Wheel Locks | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Entertainment Package | Mudguards | Premium Paint | Quick Charging Cable Package | Rear Bumper Applique | Spare tire
9693,5TDZRKEC1SS284895,Limited FWD,Wind Chill Pearl,Black leather-trimmed,50500,55499,3504,55499.000000,0.000000,Hamer Toyota,"Mission Hills, CA",328,2025-09-28 2:07:37,1500W inverter | 50 State Emissions | All Weather Floor Liners | Cargo Cross Bars | Digital rearview mirror with HomeLink universal transceiver | Emergency Assistance Kit | Entertainment Package | Illuminated Door Sills | Mudguards | Premium Paint | Rear Bumper Applique
